<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/Python/chap09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第9章 深層学習

ディープニューラルネットワークを用いて識別問題をコーディングします。Google ColabでGPUを使用するときは、「ランタイム」->「ランタイムのタイプを変更」-> ハードウェアアクセラレータ -> GPU を選びます。


深層学習用ラッパーライブラリ Keras は、ver3.0 からバックエンドとして TensorFlow, JAX, PyTorch が選べるマルチバックエンドになります。この notebook では、その pre-release である [Keras Core](https://github.com/keras-team/keras-core) でコーディングを行います。

事前のインストール

In [ ]:
!pip install keras-core transformers

## 畳み込みネットワークによる画像認識

## 準備

必要なライブラリ等を読み込みます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras_core as keras

## データの読み込み

fasion MNISTデータ（学習用60000事例、評価用10000事例、それぞれ28x28の行列）を使います。

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# 正解ラベル
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag','Ankle boot']

In [ ]:
# いくつかの画像を表示
plt.gray()
fig, axs = plt.subplots(3, 3)
for i in range(3):
  for j in range(3):
    axs[i,j].axis("off")
    axs[i,j].set_title(class_names[y_train[i*3+j]])
    axs[i,j].imshow(X_train[i*3+j])

入力を3次元テンソルから4次元テンソルに変換します。各次元は、データ数、縦のピクセル数、横のピクセル数、チャネル数（グレースケール画像は1）を表します。

In [ ]:
X_train = X_train.reshape(X_train.shape[0] , 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

値のとりうる範囲を0-255から0-1に変換します。

In [ ]:
X_train = X_train / 255.0
X_test = X_test / 255.0

まず、通常の3階層ネットワークで学習・評価します。

In [ ]:
model1 = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model1.summary()

損失関数の `sparse_categorical_crossentropy` は正解ラベルが整数で与えられているときに用います。one-hotベクトルで与えられている場合は `categorical_crossentropy`です。

In [ ]:
model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

学習時の各エポックで検証用データを使ったスコアがログに記録されます。`validation_split` を設定すると、学習用データの中から指定した割合に応じて検証用データが作成されます。

In [ ]:
model1.fit(X_train, y_train, epochs=5, batch_size=200, validation_split=0.1)

評価用データで性能を評価します

In [ ]:
test_loss, test_acc = model1.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

畳み込みネットワークで学習します。モデルのパラメータ数で構造を確認してください。

In [ ]:
model2 = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(16, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
model2.summary()

In [ ]:
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model2.fit(X_train, y_train, epochs=5, batch_size=200, validation_split=0.1)

In [ ]:
test_loss, test_acc = model2.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

Dropoutを入れます。ユニットの半数が消えている状態で学習を行うので、epochは倍の回数をとります。

In [ ]:
model3 = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])
model3.summary()

In [ ]:
model3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model3.fit(X_train, y_train, epochs=10, batch_size=200, validation_split=0.1)

評価用データで評価します。

In [ ]:
test_loss, test_acc = model3.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

## リカレントニューラルネットワークによる自然言語処理

IMDBデータは映画のレビューに対して、P/N(肯定/否定)のラベルが付いた学習データです。学習用に25000事例、評価用に25000事例用意されていて、PNの割合はそれぞれ50%です。
各レビューは単語列ではなく、単語インデックスの系列として表現されています。

ここでは、頻度上位10000語を対象とし、データの大きさは先頭の50単語に限定します。

## 準備

必要なライブラリ等を読み込みます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras_core as keras

In [ ]:
max_features = 10000
maxlen = 50
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

単語インデックスを単語に戻して、元のデータを確認します。インデックスは"padding", "start of sequence","unknown"にそれぞれ0,1,2が割り当てられているので、3つずらして対応させます。

In [ ]:
word_index = keras.datasets.imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[0]])
decoded_review

単純なRNNを構成して学習させます。

In [ ]:
model1 = keras.Sequential([
    keras.layers.Input(shape=(50,)),
    keras.layers.Embedding(max_features, 128),
    keras.layers.SimpleRNN(64),
    keras.layers.Dense(1, activation='sigmoid')
])
model1.summary()

In [ ]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model1.fit(X_train, y_train, epochs=2, batch_size=32, validation_split=0.1)

In [ ]:
test_loss, test_acc = model1.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

双方向LSTMを試します。

In [ ]:
model2 = keras.Sequential([
    keras.layers.Input(shape=(50,)),
    keras.layers.Embedding(max_features, 128),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(1, activation='sigmoid')
])
model2.summary()

In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model2.fit(X_train, y_train, epochs=2, batch_size=32,validation_split=0.1)

In [ ]:
test_loss, test_acc = model2.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.4}')

## 事前学習モデルを用いた識別

transformersライブラリを用いて、BERTでPN判定を行います。pipelineインスタンス作成時にタスクを与えると、デフォルトの設定が適用されます。また、モデルを与えると、そのモデルの設定が適用されます。あとは、そのインスタンスに入力を与えるだけで、結果が返却されます。

In [ ]:
from transformers import pipeline

nlp = pipeline('sentiment-analysis')

sentences = [
    "I love this movie, it's amazing and I had a great time.",  # ポジティブな文
    "I am having a bad experience with this story. It's borning." # ネガティブな文
]

for sentence in sentences:
    result = nlp(sentence)
    print(f"Text: {sentence}")
    print(f"Sentiment: {result}\n")